# 1. Import of input data

In [ ]:
import nbimporter
import sys
import os
from pathlib import Path

In [ ]:
# Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CoolingDryCooler.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CoolingDryCooler.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

In [ ]:
# Database parameter import
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Cooling - Dry cooler')

# 2. Input parameters

In [ ]:
# DASHBORD - VALUE INPUTS
Power = energysystem['Cooling capacity [kW]']
EnergyCooling = energysystem['Energy produced [kWh /year]']
COP = energysystem['COP']
ElectricityGrid = energysystem['Energy input from national grid [kWh/year]']
ElectricityPV = energysystem['Energy input from Photovoltaic system [kWh/year]']
ElectricityRICE = energysystem['Energy input from CHP-Internal Combustion Engine [kWh/year]']
ElectricityCombinedCycle = energysystem['Energy input from CHP-Combined cycle [kWh/year]']
ElectricityGasTurbine = energysystem['Energy input from CHP-Gas Turbine [kWh/year]']
ElectricityWtE = energysystem['Energy input from CHP-Waste-to-Energy [kWh/year]']
ElectricityORC = energysystem['Energy input from CHP-Organic Rankine Cycle [kWh/year]']

In [ ]:
# DASHBOARD - INPUT PARAMETERS
mass = -0.0027 * (Power**2) + 4.9167 * Power
Lifespan = 15 #years

# 3. Life cycle inventories

## 3.1 Module A1-3

In [ ]:
DryCooler_lowalloyedsteel = 0.84 * mass
DryCooler_aluminium = 0.07 * mass
DryCooler_chromiumsteel = 0.05 * mass
DryCooler_copper = 0.05 * mass
DryCooler_sheetrollingsteel = 0.84 * mass
DryCooler_sheetrollingaluminium = 0.07 * mass

## 3.2 Module A4

In [ ]:
DryCooler_Truk_16_32 = ((500 + 200)*(mass)/1000)
DryCooler_Light_Truk = (50*(mass)/1000)

## 3.3 Module B2

In [ ]:
DryCooler_Light_Truk_B2 = 50 * (Lifespan/2) * 0.01 * (mass/1000)

## 3.4 Module B4

In [ ]:
Lifespan = 25 #year
DryCoolerNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFDryCooler = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [ ]:
#Parameters for energy input repartition
TotalElectricityInput = (ElectricityGrid + ElectricityPV + ElectricityRICE + ElectricityCombinedCycle +
    ElectricityGasTurbine + ElectricityWtE + ElectricityORC)

DryCooler_EleGrid = ElectricityGrid / TotalElectricityInput
DryCooler_ElePV = ElectricityPV / TotalElectricityInput
DryCooler_EleRICE = ElectricityRICE / TotalElectricityInput
DryCooler_EleCombinedCycle = ElectricityCombinedCycle / TotalElectricityInput
DryCooler_EleGasTurbine = ElectricityGasTurbine / TotalElectricityInput
DryCooler_EleWtE = ElectricityWtE / TotalElectricityInput
DryCooler_EleORC = ElectricityORC / TotalElectricityInput

In [ ]:
DryCooler_usephaseelectricity = (1 / COP) * EnergyCooling * RSP * DryCooler_EleGrid * Grid
DryCooler_usephaseelectricityPVExt = (1 / COP) * EnergyCooling * RSP * DryCooler_EleGrid * PV
DryCooler_usephaseelectricityPV = (1 / COP) * EnergyCooling * RSP * DryCooler_ElePV

DryCooler_usephaseelectricityRECSPV = DryCooler_usephaseelectricity * 0.0056 * RECSPV
DryCooler_usephaseelectricityRECSHydro = DryCooler_usephaseelectricity * 0.0056 * RECSHydro
DryCooler_usephaseelectricityRECSEolic = DryCooler_usephaseelectricity * 0.0056 * RECSEolic

## 3.6 Module C2

In [ ]:
DryCooler_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [ ]:
DryCooler_EOL_aluminium = DryCooler_aluminium
DryCooler_EOL_copper = DryCooler_copper
DryCooler_EOL_chromiumsteel = DryCooler_chromiumsteel
DryCooler_EOL_lowalloyedsteel = DryCooler_lowalloyedsteel

## 3.8 Module C4

In [ ]:
DryCooler_EOL_landfill_aluminium = DryCooler_EOL_aluminium * LandfillMetals
DryCooler_EOL_landfill_copper = DryCooler_EOL_copper * LandfillMetals
DryCooler_EOL_landfill_lowalloyedsteel = DryCooler_EOL_lowalloyedsteel * LandfillMetals
DryCooler_EOL_landfill_chromiumsteel = DryCooler_EOL_chromiumsteel * LandfillMetals2

## 3.9 Module D1 (related to the export of secondary materials)

In [ ]:
DryCooler_EOL_recycling_aluminium = (RecyclingMetals * RecyclingEfficiencyMetals * DryCooler_EOL_aluminium) - (RecycledContentMetals * DryCooler_EOL_aluminium)
DryCooler_EOL_primary_aluminium = ((RecycledContentMetals * DryCooler_EOL_aluminium) - (RecyclingMetals * RecyclingEfficiencyMetals * DryCooler_EOL_aluminium)) * SRmetals

DryCooler_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * DryCooler_EOL_copper) - (RecycledContentMetals * DryCooler_EOL_copper)
DryCooler_EOL_primary_copper = ((RecycledContentMetals * DryCooler_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * DryCooler_EOL_copper)) * SRmetals

DryCooler_EOL_recycling_lowalloyedsteel = (RecyclingMetals * RecyclingEfficiencyMetals * DryCooler_EOL_lowalloyedsteel) - (RecycledContentMetals * DryCooler_EOL_lowalloyedsteel)
DryCooler_EOL_primary_lowalloyedsteel = ((RecycledContentMetals * DryCooler_EOL_lowalloyedsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * DryCooler_EOL_lowalloyedsteel)) * SRmetals

DryCooler_EOL_recycling_chromiumsteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * DryCooler_EOL_chromiumsteel) - (RecycledContentMetals * DryCooler_EOL_chromiumsteel)
DryCooler_EOL_primary_chromiumsteel = ((RecycledContentMetals * DryCooler_EOL_chromiumsteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * DryCooler_EOL_chromiumsteel)) * SRmetals

# 4. Data for export 

In [ ]:
ThermalEnergyAbsorbed_DryCooler = EnergyCooling * RSP